In [2]:
import csv
import json
import requests

In [4]:
def analyze_entities(articles):
    api_endpoint = "https://language.googleapis.com/v1/documents:analyzeEntities"
    api_key = ""  # Replace with your actual API key
    
    entity_results = []  # List to store entity analysis results
    
    for idx, article in enumerate(articles, start=1):
        text_content = article["Text Content"]
        article_title = article["Article Title"]
        
        # Create the request payload for entity analysis
        data = {
            "document": {
                "type": "PLAIN_TEXT",
                "content": text_content
            },
            "encodingType": "UTF8",
        }
        
        params = {
            "key": api_key
        }
        
        headers = {
            "Content-Type": "application/json"
        }
        
        # Send the HTTP POST request for entity analysis
        response = requests.post(api_endpoint, params=params, headers=headers, data=json.dumps(data))
        
        if response.status_code == 200:
            entity_data = response.json()
            # Get the entities from the analysis results
            entities = entity_data.get("entities", [])
            for entity in entities:
                entity_result = {
                    "Article Number": idx,
                    "Article Title": article_title,
                    "Entity Name": entity["name"],
                    "Entity Type": entity["type"],
                    "Entity Salience": entity["salience"],
                    "Entity Mentions": ', '.join([mention["text"]["content"] for mention in entity.get("mentions", [])])
                }
                entity_results.append(entity_result)
        else:
            print(f"Error for Article {idx}: {response.status_code} - {response.text}")
            continue
    
    # Save the entity analysis results to a CSV file
    csv_file = "entity_results.csv"
    csv_headers = ["Article Number", "Article Title", "Entity Name", "Entity Type", "Entity Salience", "Entity Mentions"]
    
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=csv_headers)
        writer.writeheader()
        writer.writerows(entity_results)
    
    print(f"Entity analysis results saved to {csv_file}")

if __name__ == "__main__":
    # Read articles and their text content from a CSV file
    articles = []
    with open('articles.csv', mode='r', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            articles.append({"Article Title": row['Article Title'], "Text Content": row['Text Content']})
    
    analyze_entities(articles)


Entity analysis results saved to entity_results.csv
